Hello!

Up-to-date geographical data of the Dutch municipalities and neighbourhoods ("Wijk- en buurtkaart" can be retrieved from: https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data

You go Glen Coco!


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import random
import lisualisation as lis
destination = "../layouts/partials/svgs/elan_map_raw.svg"

In [ ]:
colours = ["#e70777", "#00b0f0", "#7030a0", "#9895BC", "#9895BC", "#9895BC", "#9895BC" ,"#9895BC"]

In [ ]:
df = gpd.read_file("wijkbuurtkaart_2023_v1/gemeenten_2023_v1.shp")
df = df[df.H2O == "NEE"]
gemeentecodes = pd.read_excel("elan_classification.xlsx", sheet_name=None)

In [ ]:
gemeentecodes["classification"]

Include sexy colors

In [ ]:
def sexy_color_generator(amount):
    return ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(amount)]

gemeentecodes["classification"]["color"] = sexy_color_generator(gemeentecodes["classification"].shape[0])

Get geometries of ELAN municipalities

In [ ]:
elan = gemeentecodes["classification"].merge(df, left_on="gm_code", right_on="GM_CODE")
elan = gpd.GeoDataFrame(elan)

Get geometries of province

In [ ]:
provincedf = pd.read_excel("Gemeenten alfabetisch 2023.xlsx")
provincedf = provincedf[provincedf.ProvinciecodePV == "PV28"]
province = df[df.GM_CODE.isin(provincedf.GemeentecodeGM)]

Generate background

In [ ]:
background = elan.dissolve(by='class', aggfunc = 'sum')
# background["color"] = background.index
# background["color"].replace(sexy_colors, inplace=True)
background["color"] = sexy_color_generator(background.shape[0])

Create visual

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

#province.plot(ax=ax, color='#ffffff', edgecolor='#001157', linewidth=1)
background.plot(ax=ax, color=background["color"], linewidth=0)
elan.plot(ax=ax, color=elan["color"], linewidth=0)

_=ax.axis('off')
#plt.show()
plt.tight_layout(pad=0, w_pad=0, h_pad=0)
plt.savefig(destination, dpi=300, format="svg", transparent=True, bbox_inches= 'tight', pad_inches = 0)

Assign all the paths in the generated svg image the aprropriate class to enable interaction and animation. We use the colors to match up the geometry and the right class. We also include style attributes and make it interactive.

In [ ]:
background["myclass"] = "region"+ background.index.astype(str)
elan["myclass"] = elan.Naam
combi_df = pd.concat([background, elan])

In [ ]:
css= r'''
    .category1, .region1{
        fill: #e70777;
    }
    
    .category2, .region2{
        fill: #00b0f0;
    }

    .category3, .region3{
        fill: #7030a0;
    }

    svg{
        font-family: "Open Sans", sans-serif;
        font-size: 30px;
    }

    .circle_stat{
        transform-box: fill-box;
        transform-origin: center;
        transform: rotate(-90deg);
    }

    .stat_title{
        font-weight: bold;
        fill: #786ebd;
    }
    
    .category1:hover, .category2:hover, .category3:hover{
        filter: brightness(1.25);
    }

    .hover_text{
        font-size: 0.9em;
        fill: #6a6874;
    }
    
    .hover_bar{
        fill: #FFFFFF;
    }

    .hover_container_hidden{
        display: none;
    }
    '''

`//<![CDATA[` and `//]]>` are required since that allows for proper parsing

In [ ]:
js = r"""
<script type="text/javascript">   
//<![CDATA[    
    const cat_names = ["1", "2", "3"];
    const cats = cat_names.map(cat => document.getElementsByClassName("category" + cat));
    const regs = cat_names.map(cat => document.getElementById("hover_container_" + cat));
    const hovers = cat_names.map(cat => document.getElementById("hover_text_" + cat));

    for(let j = 0; j < cat_names.length; j++) {
        for(let i = 0; i < cats[j].length; i++) {
            cats[j][i].onmouseleave = function(){
                regs[j].classList.add('hover_container_hidden');
                hovers[j].children[0].textContent = '';
                hovers[j].children[1].textContent = '';
            };
            
            cats[j][i].onmouseover = function(){
                hovers[j].children[0].textContent = 'Municipality: ';
                console.log(cats[j][i].getAttribute('data-name'));
                hovers[j].children[1].textContent = cats[j][i].getAttribute('data-name');
                regs[j].classList.remove('hover_container_hidden');
            };
        };
    };    
        
    //]]>
</script>   
"""

Hoverinfo for les regions

In [ ]:
stats = gemeentecodes["stats"]
stats_2024 = stats[stats["year"] == 2024].copy()

In [ ]:
practices = stats_2024.sort_values("region")["practices"].tolist()
patients = stats_2024.sort_values("region")["patients"].tolist()

In [ ]:
hover_regions ="<g>"
#pink, blue, purple
x = [410, 315, 385]
y = [90, 300, 420]
hover_width = 320
x_text_l_margin = 5
hover_height = 80

for i in range(3):
    hover_regions += rf'''
        <g id="hover_container_{i+1}" class="hover_container_hidden">
            <rect class="hover_bar" x="{x[i]}" width="{hover_width}" y="{y[i]}" height="{hover_height}" rx="15" />
            <text id="hover_text_{i+1}" class="hover_text" x="{x[i]}" y="{y[i]}" text-anchor="left" dy=".3em"><tspan x="{x[i] + x_text_l_margin}" dy="1.2em"></tspan><tspan x="{x[i] + x_text_l_margin}" dy="1.2em"></tspan></text>
        </g>
    '''
hover_regions += "</g>"

In [ ]:
with open(destination, 'r') as file:
    # read a list of lines into data
    lines = file.readlines()

lines = lines[3:]

lines[0] = '''<svg xmlns:xlink="http://www.w3.org/1999/xlink" class="{{ .class }}" viewBox="0 0 720 528.173872" xmlns="http://www.w3.org/2000/svg" version="1.1">'''

munic_colors = elan["color"].tolist()

for i in range(0, len(lines)):
    line = lines[i]
    if ("fill" in line) and ("path" in line):
        line_split = line.split('style="fill: ', 1)
        color = line_split[1][:7].upper()
        entry = combi_df[combi_df.color == color]
        my_class = entry["myclass"].item()
        if color in munic_colors:
            #class holds a lot of classes, data-name is the name of the municipality
            my_class += " municipality category" + str(int(entry["class"].item())) + '" data-name="' + my_class
        lines[i] = line_split[0] + 'class="' + my_class + '"/>'
    elif "</style>" in line: #add css
        line_split = line.split('</style>', 1)
        lines[i] = line_split[0] + css + "</style>"
    elif "</svg>" in line: # add hover feature
        lines[i] = hover_regions + js + line

with open("../layouts/partials/svgs/elan_map.svg", 'w') as file:
    file.writelines(lines)

to generate circles

In [ ]:
lis.make_sexy_circle(stats_2024["patients"].tolist(), colours, "../layouts/partials/svgs/patients.svg")
lis.make_sexy_circle(stats_2024["practices"].tolist(), colours, "../layouts/partials/svgs/practices.svg")